# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd

from nltk import word_tokenize, download as nltk_download
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sqlalchemy import create_engine

pd.set_option('display.max_columns', 200)

nltk_download('punkt')
nltk_download('stopwords')
nltk_download('wordnet')

[nltk_data] Downloading package punkt to /home/brendalf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brendalf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brendalf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine, index_col="id")
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
X = df.message.values
X[:6]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-'], dtype=object)

In [8]:
Y = df[df.columns[4:]].values
Y[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    # normalize the text removing ponctuation and making it lowercase
    normalized = re.sub("[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize the normalized text
    words = word_tokenize(normalized)
    
    # remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # lemmetatization
    wordnet = WordNetLemmatizer()
    words = [wordnet.lemmatize(w) for w in words]
    
    # stemming
    porter = PorterStemmer()
    words = [porter.stem(w) for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [12]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_pred = pipeline.predict(X_test)

In [14]:
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test[:, i], Y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5400
           1       0.82      0.47      0.60      1154

    accuracy                           0.89      6554
   macro avg       0.86      0.72      0.77      6554
weighted avg       0.88      0.89      0.88      6554

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

              precision    recall  f1-score   support

           0       0.79      0.86      0.83      3819
           1       0.78      0.69      0.73      2735

    accuracy                           0.79      6554
   macro avg       0.79      0.77      0.78      6554
weighted avg       0.79      0.79      0.79      6554

              preci

/home/brendalf/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6460
           1       0.82      0.10      0.17        94

    accuracy                           0.99      6554
   macro avg       0.90      0.55      0.58      6554
weighted avg       0.98      0.99      0.98      6554

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6395
           1       0.88      0.04      0.08       159

    accuracy                           0.98      6554
   macro avg       0.93      0.52      0.54      6554
weighted avg       0.97      0.98      0.97      6554

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6486
           1       1.00      0.01      0.03        68

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

              preci

In [15]:
scores = []
precision = []
recall = []

for i in range(Y_test.shape[1]):
    scores.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['f1-score'])
    precision.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['precision'])
    recall.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['recall'])

print("f1 score: {:.5f}".format(sum(scores)/len(scores)))
print("precision: {:.5f}".format(sum(precision)/len(precision)))
print("recall: {:.5f}".format(sum(recall)/len(recall)))

f1 score: 0.93976
precision: 0.94236
recall: 0.95173


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=4, verbose=5)

In [11]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 17.6min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 98.5min
/home/brendalf/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 256.3min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 632.0min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 922.0min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 1417.9min
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed: 1792.2min
[Parallel(n_jobs=4)]: Done 972 out of 972 | elapsed: 2077.7min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [38]:
cv.best_estimator_['vect']

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.75, max_features=10000, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f2f9e4c09e0>,
                vocabulary=None)

In [37]:
cv.best_estimator_['tfidf']

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [41]:
cv.best_estimator_['clf']

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
Y_pred = cv.predict(X_test)

In [30]:
for i in range(Y_test.shape[1]):
    print(classification_report(Y_test[:, i], Y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5431
           1       0.83      0.49      0.61      1123

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      3851
           1       0.75      0.70      0.73      2703

    accuracy                           0.78      6554
   macro avg       0.78      0.77      0.77      6554
weighted avg       0.78      0.78      0.78      6554

              preci

/home/brendalf/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
scores = []
precision = []
recall = []

for i in range(Y_test.shape[1]):
    scores.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['f1-score'])
    precision.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['precision'])
    recall.append(classification_report(Y_test[:, i], Y_pred[:, i], output_dict=True, zero_division=0)['weighted avg']['recall'])

print("f1 score: {:.5f}".format(sum(scores)/len(scores)))
print("precision: {:.5f}".format(sum(precision)/len(precision)))
print("recall: {:.5f}".format(sum(recall)/len(recall)))

f1 score: 0.94387
precision: 0.94445
recall: 0.95369


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [42]:
model = cv.best_estimator_

with open('../models/model.pkl', "wb") as f:
        pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.